In [250]:
import numpy as np
import math

In [384]:
class MakeReceivedImg(object):
    def __init__(self, numberOfLEDs=16, gaussSigma=0.45, kernelSize=9, maxLum=1, minLum=0, boxNoise=0.3, offset=False):
        """
        Initial functions of this class
        """
        self.numberOfLEDs = numberOfLEDs
        self.sqrtnumberofled = math.sqrt(numberOfLEDs)
        self.gaussSigma = gaussSigma
        self.kernelSize = kernelSize
        self.maxLum = maxLum
        self.minLum = minLum
        self.boxNoise = boxNoise
        self.offset = offset
        if offset:
            self.add_led = 1
        else:
            self.add_led = 0
    
    def RandomLEDs(self):
        """
        Making randomly blinking LED and store them in array(numpy) 
        numberOfLEDs: Number of LEDs. Default = 16
        max_lum_value: maximum luminance value of LEDs
        min_lum_value: minimum luminance value of LEDs
        """
        # Generating random integers [0,1]
        blinking_leds = np.random.randint(0, 2, self.numberOfLEDs)
        
        # aplly max/min luminance value depending on its blinking condition
        for index, value in enumerate(blinking_leds):
          if value == 0:
            blinking_leds[index] = self.minLum
          else:
            blinking_leds[index] = self.maxLum

        # [sqrt(len(blinking_leds)]
        led_len = math.sqrt(len(blinking_leds))
        assert led_len % 2 == 0, 'Expected: 0 == led_len%2 \n Actual: 1 == led_len%2'

        # change dtype float to int
        led_len = int(led_len)

        # reshape led_len to two-d array from one-d array
        led_condition = np.reshape(blinking_leds, (led_len, led_len))

        return np.array(blinking_leds)


    def GaussChannelAndInv(self):
        """
        Creating channel parameters based on Gauss function
        """

        y_scale = int(self.sqrtnumberofled)
        x_scale = int(self.sqrtnumberofled)        

        # 配列の確保
        gauss_channel = [[0 for i in range(y_scale * x_scale)] for j in range(y_scale * x_scale)]

        # 自身と他の画素から影響を受ける画素の決定
        for y in range(0, y_scale):
            for x in range(0, x_scale):
                target_pixel = y * x_scale + x  

                # target_pixelへ影響を与えるinfluence_pixelを決定
                for i in range(y_scale):
                    for j in range(x_scale):

                        influence_pixel = i * x_scale + j

                        # ガウシアンフィルの作成
                        distance = (j - x) * (j - x) + (i - y) * (i - y)
                        weight = math.exp(-distance / (2.0 * self.gaussSigma * self.gaussSigma))  * (1 / (2.0 * math.pi * self.gaussSigma * self.gaussSigma))

                        if ((j - x) * (j - x)) >= self.kernelSize or ((i - y) * (i - y)) >= self.kernelSize:
                            weight = 0
                        else:
                            gauss_channel[target_pixel][influence_pixel] = weight

                gauss_channel = np.array(gauss_channel)

        # 逆行列の作成
        inv_channel = gauss_channel.copy()
        inv_gauss_channel = np.linalg.inv(inv_channel)
        return gauss_channel, inv_gauss_channel
      
      
    def Filtering(self, leds):
        """
        Aplly filtering to LED arrays created in RandomLEDs
        """

        channel, invchannel = self.GaussChannelAndInv()
        #pixel_values = np.zeros(self.numberOfLEDs)

        pixel_values = np.dot(channel, leds)

       # pixel_values = np.dot(invchannel, pixel_values)

        return pixel_values
 
        """  
        def InvGaussChannel(self):

        #逆行列を施す


        channel = self.GaussChannel()
        inv_gauss_channel = np.linalg.inv(channel)
        return inv_gauss_channel
            """      
  
  
    def GetNoise(self):
        """
        Generate noise by boxmuller method
        """

        from scipy.stats import uniform

        # 独立した一様分布からそれぞれ一様乱数を生成
        np.random.seed()
        N = self.numberOfLEDs
        rv1 = uniform(loc=0.0, scale=1.0)
        rv2 = uniform(loc=0.0, scale=1.0)
        U1 = rv1.rvs(N)
        U2 = rv2.rvs(N)

        # Box-Mullerアルゴリズムで正規分布に従う乱数に変換
        # 2つの一様分布から2つの標準正規分布が得られる
        X1 = np.sqrt(-2*np.log(U1)) * np.cos(2*np.pi*U2) * self.boxNoise
        #X2 = np.sqrt(-2 * np.log(U1)) * np.sin(2 * np.pi * U2) * boxNoise

        return X1
    
    def ReceivedImg(self):
        """
        Create received image
        """
        leds = self.RandomLEDs()
        pix = self.Filtering(leds)
        noise = self.GetNoise()
        
        if self.offset:
           # offset = np.random.randint(10, 11, self.numberOfLEDs)
            receivedimg = pix + noise + 10*self.maxLum*0.01
            leds = np.append(leds, 1)
        else:
            receivedimg = pix + noise
            
        return receivedimg, leds
    
    def create_dataset(self, loop=100):
        """
        Create dataset
        """
        
        # LED condition (1/0)
        led_condition = np.empty((0))
        pixel_value = np.empty((0))
        
        for iteration in range(loop):
            receivedimg, led_condi = self.ReceivedImg()
            pixel_value = np.hstack((pixel_value, receivedimg))
            
            led_condition = np.hstack((led_condition, led_condi))
           
        led_condition_2d = np.reshape(led_condition, (loop, self.numberOfLEDs+self.add_led))
        pixel_value_2d = np.reshape(pixel_value, (loop, self.numberOfLEDs))
        
        return led_condition_2d, pixel_value_2d

if __name__ == '__main__':
    mri = MakeReceivedImg()

In [444]:
mri = MakeReceivedImg(gaussSigma=0.3, boxNoise=0.2, offset=True)
led, pix = mri.create_dataset(loop=1000)
print(pix.shape)
print(led.shape)

(1000, 16)
(1000, 17)


In [445]:
def estimate_channel(led_pattern, pixel_value, offset=False):
    """
    提案手法でのチャネル推定
    """
    
    #led, pix = mri.create_dataset(loop=10)
    inv_led = np.linalg.pinv(led_pattern)
    len_loop, len_led = led_pattern.shape
    
    if offset:
        total_led = len_led - 1
    else:
        total_led = len_led

    estimated_channel = np.array([[0 for i in range(len_led)] for j in range(total_led)], dtype='float32')
    
    for idx_led in range(total_led):
        # indx_led番目のLEDの点滅状態を格納
        tmp_led = []
        for idx_loop in range(len_loop):
            tmp_led.append(pixel_value[idx_loop][idx_led])
        # idx_led番目のLEDのチャネルを格納
        tmp_channel = np.dot(inv_led, tmp_led)
        for i in range(len_led):
            estimated_channel[idx_led][i] = tmp_channel[i]
            
    return np.array(estimated_channel, dtype='float32')

In [446]:
def estimate_channel_conv(numleds=16, numimages=50):
    """
    個別点灯画像の1000枚の平均を用いたチャネル推定
    """
    estimated_channel = np.array([[0 for i in range(numleds)] for j in range(numleds)], dtype='float32')
    for led in range(numleds):
        leds = np.zeros(numleds, dtype='int32')
        leds[led] = 1
        total_pix_val = [0]
        for image in range(numimages):
            pix = mri.Filtering(leds)
            noise = mri.GetNoise()
            pix_val = pix + noise + 10*0.01
            total_pix_val += pix_val / numimages
        
        for i in range(numleds):
            estimated_channel[i][led] = total_pix_val[i]
    
    return estimated_channel

In [447]:
ch_pro = estimate_channel(led, pix, offset=True)
ch_conv = estimate_channel_conv()

In [448]:
pixel_values = pix
answer_signals = led
offset_value = ch_pro[:, 16]
print(offset_value)

[0.07322998 0.11116165 0.10536639 0.15135916 0.12157221 0.1524059
 0.09756571 0.13025825 0.10300929 0.07000576 0.12860724 0.0932155
 0.130652   0.10387751 0.176376   0.08244157]


In [449]:
def modulation_zf(pixelValue, answers, gauss, loop_times, offset=False):
    """
    推定したチャネルからZFにより復調を行う
    """
    _, len_led = pixelValue.shape
    
    if offset:
        offset_value = gauss[:, 16]
    
    answers = answers[:, :16]
    gauss = gauss[:, :16]
    inv_gauss = np.linalg.pinv(gauss)

    error = 0
    
    for loop in range(loop_times):
        
        pixel_value = pixelValue[loop][:]
        
        if offset:
            pixel_value = pixel_value - offset_value
        
        answer = answers[loop][:]
        estimated_signals = np.dot(inv_gauss, pixel_value)
        estimated_signals_copy = estimated_signals.copy()
        estimated_signals_copy = np.where(estimated_signals_copy > 0.5, 1, 0)
        error += np.sum(estimated_signals_copy != answer)
    
    print(answer.shape, estimated_signals_copy.shape)
        
    return error / (loop_times*len_led)

In [450]:
ber_pro = modulation_zf(pixel_values, answer_signals, ch_pro, 1000, offset=True)
ber_conv = modulation_zf(pixel_values, answer_signals, ch_conv, 1000)
print(ber_pro)
print(ber_conv)

(16,) (16,)
(16,) (16,)
0.0
0.008375


In [451]:
print(ch_pro.shape)

(16, 17)
